# This notebook prepares the primary data sets and visits some tips and techniques along the way

* Full data pulled from retrosheet.org
* Chadwick tools used for converting retrosheet data http://chadwick.sourceforge.net/doc/cwtools.html

We'll be using Retrosheet baseball data for our examples in this discussion.  At the center of the work will be the Retrosheet 'Event Files' from 2017.  Each row in the event file describes some kind of action on the field. Any time the game situation is different from the previous pitch, one or more events is created.  The events are rich in data.  They indicate the game, the inning, the batter, the type of hit, the direction of the hit, the number of runners on base etc. We'll take a look at the event codes a bit later to get some impression of the depth of information.  There are 191,996 events recorded for 2017.  There were 81\*30 or 2430 scheduled games in 2017. This yields, on average, about 79 events per game.

In [ ]:
import pandas as pd
import os
import matplotlib

# % and %% are ipthyon 'magics'
# ! is ipython's shell execution shortcut

In [2]:
%history

%history
%history


In [ ]:
!ls data_public/*.EV*

In [ ]:
%%sh
head -3 data_public/2017CHA.EVA
echo ""
echo 'data_public/2017CHA.EVA'
echo ""
sort -k2 -t, data_public/2017CHA.EVA |head -5

#### Useful techniques for interacting with the shell

* <b>Use assignment to capture the ouput of your ! command</b>

In [ ]:
files = !ls

In [ ]:
type(files)

In [ ]:
files.grep('\.i.*')

In [ ]:
files.p

## Passing Python output to the shell
#### Here we'll do the opposite and more powerful technique
* <b>We'll use Use {expression} to pass from ipython to the shell</b>

In [ ]:
extension = 'ipynbb'

In [ ]:
!ls *.{extension}

* Hmm... Extra trailing 'b'. Lets take a slice of the extension string

In [ ]:
!ls *.{extension[0:-1]}

### This shows us a few techniques and examples. <br> In the following section, we'll use these techniques to bring in the data set that we'll use for our analysis.

### Here we're preparing the file.  Run the cwevent executable with -n and capture the header

In [ ]:
#Chadwick expects a 'team' file in the cwd
!ln -s ./data_public/TEAM2017 team

In [ ]:
! cwevent -n data_public/2017SEA.EVA |head -1 >data_public/atbats.txt

### Now we'll shell out and run a loop to invoke the converter on each event file.  We'll also concatentate the roster files in a separate command

In [ ]:
%%sh
for x in $(ls -1 data_public/*.EV*); do cwevent $x >>data_public/atbats.txt; done
cat data_public/*.ROS >data_public/rosters.txt

### Standard Python file to dictionary
* Constants for event codes
* Here we're reading a two column file into a Python key:val (dictionary) data structure
* This allows us to use '2B' and 'HR' vs 21 and 23 to identify event rows of interest

In [ ]:
event_code = {}
with open("data_public/event_codes.txt") as f:
    for line in f:
       (val, key) = line.split()
       event_code[key] = int(val)

In [ ]:
event_code

In [ ]:
df_events=pd.read_csv('./data_public/atbats.txt')
df_players=pd.read_csv('./data_public/players.txt')

### Here are a few Pandas tools for getting an overview of a dataframe

In [ ]:
df_events.shape

In [ ]:
df_events.columns

In [ ]:
df_events.head()

In [ ]:
df_events.info()

### Now we'll look at a few pandas techniques
* First we'll restrict the dataframe to a single column
* Next we'll restrict the dataframe to a set of columns
* Third we'll breakdown the contants of a column
* Fourth we'll use value_counts() to get a summary

In [ ]:
df_events['GAME_ID'].head()

* Notice that we restrict with dataframe[] and provide a python list of the colums with ['item1','item2'....] resulting in doubling of the brackets

In [ ]:
df_events[['GAME_ID','AWAY_TEAM_ID','BALLS_CT','RESP_BAT_ID','OUTS_CT','EVENT_CD']].head()

In [ ]:
df_events['GAME_ID'].str[0:3].head()

Below we apply the series.value_counts( ) method to return a series whose index is each distinct value 

In [ ]:
df_events['GAME_ID'].str[0:3].value_counts()

### Let's look at the data quickly to get a bit of a better idea of what's in the files

In [ ]:
df_events[['GAME_ID','AWAY_TEAM_ID','BALLS_CT','RESP_BAT_ID','OUTS_CT','EVENT_CD']].head()

* Note above that there are two consecutive events for the same batter (cruzn002).  We'll use the pandas indexed lookup method (.iloc) to convert a row of the dataframe into a series whose index is the column name and values are the data from that row. Event codes here are 14=walk, 3=K, 2=generic out and 4=stolen base

In [ ]:
df_events.iloc[3].loc['EVENT_CD']

In [ ]:
type(df_events['AWAY_TEAM_ID'].value_counts())

In [ ]:
df_events['AWAY_TEAM_ID'].value_counts().index

#### This shows us a few techniques and examples. <br> In the following section, we'll use these techniques to bring in the data set that we'll use for our analysis.

#### Using value_counts to inspect a column
* Here we use value_counts to generate list of entities from a series 
* Functionally identical to "<b> cat file |cut -d, -f4|sort|uniq -c |sort -rn</b>"
* Functionally identical to "<b>select column, count(column) group by column order by count(column)</b>")
* value_counts() is  a series or index method. It returns a series 
* Note again that the index of the series is the value in the column

In [ ]:
df_events['AWAY_TEAM_ID'].value_counts()

#### Create a few filters to restrict rows (filter is not a defined Pandas term)
* These types of restrictions return a pandas series
* We've named the series reflecting the filter(s) applied 
* Each series is a set of booleans indexed identically to the source

In [ ]:
flt_homers = df_atbats['EVENT_CD'] == event_code['HR']
flt_redsox = (df_atbats['GAME_ID'].str.startswith('BOS')) | (df_atbats['AWAY_TEAM_ID'] == 'BOS')
flt_yankees = (df_atbats['GAME_ID'].str.startswith('NYA')) | (df_atbats['AWAY_TEAM_ID'] == 'NYA')

In [ ]:
flt_homers.value_counts()

In [ ]:
flt_redsox.value_counts()

### Using an index along with python looping
* An index is an iterable object
* We traverse it here to make dictionaries of filters
* First we'll show the index, then we'll iterate to create more filters

In [ ]:
df_events['AWAY_TEAM_ID'].value_counts().index

In [ ]:
flt_home_team = {}
flt_away_team = {}
for team in df_atbats['AWAY_TEAM_ID'].value_counts().index:
    flt_home_team[team] = (df_atbats['GAME_ID'].str.startswith(team)) & (df_atbats['BAT_HOME_ID'] == 1)
    flt_away_team[team] = (df_atbats['AWAY_TEAM_ID'] == team) & (df_atbats['BAT_HOME_ID'] == 0)

* Our first practical question... How many doubles did the Red Sox have in 2017?

In [ ]:
df_atbats[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_atbats['EVENT_CD'] == event_code['DOUBLE'])].shape

* Who had them??

In [ ]:
df_atbats[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_atbats['EVENT_CD'] == event_code['DOUBLE'])]['RESP_BAT_ID']

In [ ]:
for filter in filters:
    print(filter)
    print(df_events[filters[filter] & flt_homers]['BAT_HOME_ID'].value_counts())

In [ ]:
df_events['GAME_ID'].str.startswith('BOS').value_counts()

## Pandas Merging Data Sets
* Let's revisit the Red Sox Doubles

In [ ]:
type(df_atbats[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_atbats['EVENT_CD'] == event_code['DOUBLE'])]['RESP_BAT_ID'])

In [ ]:
type(df_atbats[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_atbats['EVENT_CD'] == event_code['DOUBLE'])])

* Pandas merge() is a dataframe method so we'll have to use our dataframe version of the result set

In [ ]:
df_players.head(3)

In [ ]:
df_atbats[((flt_home_team['BOS']) | (flt_away_team['BOS'])) \
          & (df_atbats['EVENT_CD'] == event_code['DOUBLE'])]\
.merge(df_players,left_on = 'RESP_BAT_ID', right_on = 'ID')['Last'].value_counts()

## Pandas Grouping

In [ ]:
grp_teams_homers = df_events.groupby([df_events['GAME_ID'].str[0:3], df_events['AWAY_TEAM_ID'], df_events['EVENT_CD']==23])

In [ ]:
grp_teams_homers['EVENT_CD'].count()

In [ ]:
grp_away = df_events.groupby(df_events['AWAY_TEAM_ID'])
grp_home = df_events.groupby(df_events['GAME_ID'].str[0:3])

In [ ]:
grp_away[df_events[df_events['EVENT_CD'] == 23]].count()

In [ ]:
df_events[df_events['EVENT_CD'] == 23]['EVENT_CD']

In [ ]:
df_events[df_events['EVENT_CD'] == 23]['RESP_BAT_ID'].value_counts()

In [ ]:
%matplotlib inline

In [ ]:
grp_home['EVENT_CD'].head()

In [ ]:
df_events[filters['NYA']

In [ ]:
grp_hit_type=df_events.groupby('EVENT_CD')

In [ ]:
grp_hit_type.describe()

In [ ]:
for grpname,grprec in grp_hit_type:
    print(grpname)
    print(grprec)

In [ ]:
df_events[df_events['EVENT_CD'] == 23].groupby([df_events['EVENT_CD'] == 23]['EVENT_CD'])

In [ ]:
df_events[df_events['EVENT_CD' == 23]].groupby('BAT_HOME_ID').min()

In [ ]:
%%matplotlib inline